In [1]:
pip install pycvesearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pycvesearch import CVESearch
cve = CVESearch('https://cve.circl.lu')
response = cve.id('CVE-2014-0160')

In [4]:
if response:
  date = response['Published']
  print(date)
  print(type(date))

2014-04-07T22:55:00
<class 'str'>


In [ ]:
from datetime import datetime
cve_date = datetime.fromisoformat('2014-04-07T22:55:00').astimezone(timezone.utc)
cve_date = cve_date.strftime('%Y-%m-%d %H:%M:%S')
#tweet_date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
#print(tweet_date)

In [ ]:
print(type(cve_date))

<class 'str'>


In [ ]:
cve_date

In [ ]:
tweet_date = parse('2021-01-07 08:09:04')

In [ ]:
tweet_date

datetime.datetime(2021, 1, 7, 8, 9, 4)

In [ ]:
response

In [6]:
import pandas as pd
import urllib3
import time
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from dateutil.parser import parse
from pycvesearch import CVESearch
from datetime import timezone
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
cve = CVESearch('https://cve.circl.lu')
# to calculate analysis time
start_time = time.time()

# Field names for the dataset
fields_name = ['tweet_date', 'account', 'ioc_type', 'ioc_value', 'type_of_attack', 'tweet_url', 'text','cve_status','cve_first_seen', 'first_report', 'time_difference', 'time_difference(sec)']
df = pd.read_csv ('/content/drive/MyDrive/Twitter_Work_Code/TIP/CVE/cve_output.csv',names=fields_name, engine="python")
# To collect the values from malwarebazar and update the dataframe
def report(index):
    ioc_type=df.loc[index,'ioc_type']
    ioc = df.loc[index,'ioc_value']
    status = df.loc[index,'cve_status']
    if ioc_type not in ['CVE']:
      return
    if status in ['Found','Not Found']:
    	return
    source_date = df.loc[index, 'tweet_date']
    print(source_date)
    souce_date =source_date[:19]
    # To get the api response   
    while True:
    	try:
    		response = cve.id(ioc)
    		break
    	except Exception as e:
    		print({'Error':str(e)})
    if response:
      df.loc[index, 'cve_status'] = "Found"
      date = response['Published']
      cve_date = datetime.fromisoformat(date).astimezone(timezone.utc)
      cve_date = cve_date.replace(tzinfo=None)
      #cve_date.strftime('%Y-%m-%d %H:%M:%S')
      print("cvedate",cve_date)
      tweet_date = datetime.strptime(souce_date,'%Y-%m-%d %H:%M:%S')
      print("tweetdate",tweet_date)
      df.loc[index, 'cve_first_seen'] = cve_date
      time_diff = tweet_date-cve_date
      print("diff",time_diff)
      df.loc[index, 'time_difference'] = time_diff
      print(df.loc[index, 'time_difference'])
      df.loc[index, 'time_difference(sec)'] = time_diff.total_seconds()
      print(df.loc[index, 'time_difference(sec)'])
      if cve_date < tweet_date:
        df.loc[index, 'first_report'] = "NVD"
      else:
        df.loc[index, 'first_report'] = "Twitter"
      print(df.loc[index, 'first_report'] )
    else:
      df.loc[index, 'cve_status'] = "Not Found"
    print(df.loc[index, 'cve_status'])
    print(index, ioc, df.loc[index, "cve_status"])
# To run the queries as multithreaded for faster execution rate
with ThreadPoolExecutor(max_workers=1) as executor:
    executor.map(report, range(0,len(df)))
    executor.shutdown(wait=True)
print(df)

# Writeout the database as csv for storage
df.to_csv('/content/drive/MyDrive/Twitter_Work_Code/TIP/CVE/cve_output.csv', na_rep='', index=None,errors='ignore')

print(f'\nTime : {time.time() - start_time : .2f}')


Streaming output truncated to the last 5000 lines.
Found
69679 CVE-2021-29629 Found
2021-05-27 17:09:04+00:00
cvedate 2021-05-14 21:15:00
tweetdate 2021-05-27 17:09:04
diff 12 days, 19:54:04
12 days, 19:54:04
1108444.0
NVD
Found
69689 CVE-2020-24119 Found
2021-05-27 19:09:05+00:00
cvedate 2021-05-27 13:15:00
tweetdate 2021-05-27 19:09:05
diff 5:54:05
5:54:05
21245.0
NVD
Found
69707 CVE-2021-33200 Found
2021-05-27 20:08:28+00:00
cvedate 2021-05-11 19:15:00
tweetdate 2021-05-27 20:08:28
diff 16 days, 0:53:28
16 days, 0:53:28
1385608.0
NVD
Found
69712 cve-2021-31166 Found
2021-05-27 20:09:04+00:00
cvedate 2021-05-26 21:15:00
tweetdate 2021-05-27 20:09:04
diff 22:54:04
22:54:04
82444.0
NVD
Found
69714 CVE-2020-25724 Found
2021-05-27 21:09:05+00:00
cvedate 2021-05-26 17:15:00
tweetdate 2021-05-27 21:09:05
diff 1 day, 3:54:05
1 day, 3:54:05
100445.0
NVD
Found
69716 CVE-2021-20487 Found
2021-05-28 10:09:05+00:00
cvedate 2021-06-07 14:15:00
tweetdate 2021-05-28 10:09:05
diff -11 days, 19:54:05